# import libs

In [33]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from tabulate import tabulate
# %matplotlib inline
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"
from copy import deepcopy

# upload and investigate data

In [2]:
from google.colab import drive
drive.mount('/content/drive')
train = pd.read_csv('/content/drive/MyDrive/Business_hse/HW_3/train.csv', delimiter=',')
test = pd.read_csv('/content/drive/MyDrive/Business_hse/HW_3/test.csv', delimiter=',')


# connvert date-column to data format
train['period_start_dt'] = pd.to_datetime(train['period_start_dt'], format= "%Y-%m-%d")
train

Mounted at /content/drive


,Unnamed: 0,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,40370,309,2016-12-19,64.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,40372,309,2016-12-19,32.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,40373,309,2016-12-19,10.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,46272,309,2016-12-19,15.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,NaN,0.0,0.0,1000.00,1000.0,0.0,1.0
35340,35538,40372,1380,2019-12-30,NaN,0.0,0.0,2000.00,2000.0,0.0,1.0
35341,35539,40373,1380,2019-12-30,NaN,0.0,0.0,3000.00,3000.0,0.0,1.0
35342,35540,46272,1380,2019-12-30,NaN,1.0,0.0,284.29,199.0,0.0,1.0


In [3]:
def columnValues(df):
    row = [df.nunique(), (df == 0).sum(axis=0), df.isna().sum(), round(df.isna().sum() / len(df) * 100, 1), df.dtypes]
    return row

data = []
for column in train:
    data.append([column] + columnValues(train[column]))

print(tabulate(data, headers=['Column', 'Count Unique', 'Count Zeros', 'Count NaNs','% of NaNs', 'data type'], tablefmt='orgtbl'))

| Column            |   Count Unique |   Count Zeros |   Count NaNs |   % of NaNs | data type      |
|-------------------+----------------+---------------+--------------+-------------+----------------|
| Unnamed: 0        |          35344 |             1 |            0 |         0   | int64          |
| product_rk        |              6 |             0 |            0 |         0   | int64          |
| store_location_rk |             41 |             0 |            0 |         0   | int64          |
| period_start_dt   |            159 |             0 |            0 |         0   | datetime64[ns] |
| demand            |           1806 |          5454 |         1200 |         3.4 | float64        |
| PROMO1_FLAG       |              3 |         28323 |          185 |         0.5 | float64        |
| PROMO2_FLAG       |              1 |         35159 |          185 |         0.5 | float64        |
| PRICE_REGULAR     |            229 |             0 |         1127 |         3.2 | float64

In [4]:
# investigate demand driver columns (explanatory variables )

for column in train:
  if train[column].nunique() < 4:
    print(f'{column}: {list(train[column].unique())})')

PROMO1_FLAG: [np.float64(nan), np.float64(1.0), np.float64(0.0), np.float64(2.0)])
PROMO2_FLAG: [np.float64(nan), np.float64(0.0)])
NUM_CONSULTANT: [np.float64(nan), np.float64(0.0)])
AUTORIZATION_FLAG: [np.float64(nan), np.float64(1.0), np.float64(0.0)])


In [5]:
# remove those, which have only one unique value (not empty) (they do not provide any benefit when training the model)

train = train.drop(['PROMO2_FLAG', 'NUM_CONSULTANT'], axis=1)
train

,Unnamed: 0,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN
1,1,40370,309,2016-12-19,64.0,NaN,NaN,NaN,NaN
2,2,40372,309,2016-12-19,32.0,NaN,NaN,NaN,NaN
3,3,40373,309,2016-12-19,10.0,NaN,NaN,NaN,NaN
4,4,46272,309,2016-12-19,15.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,NaN,0.0,1000.00,1000.0,1.0
35340,35538,40372,1380,2019-12-30,NaN,0.0,2000.00,2000.0,1.0
35341,35539,40373,1380,2019-12-30,NaN,0.0,3000.00,3000.0,1.0
35342,35540,46272,1380,2019-12-30,NaN,1.0,284.29,199.0,1.0


In [6]:
# rename "Unnamed: 0" to id (it's needed to manage train/and)
train.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
train

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN
1,1,40370,309,2016-12-19,64.0,NaN,NaN,NaN,NaN
2,2,40372,309,2016-12-19,32.0,NaN,NaN,NaN,NaN
3,3,40373,309,2016-12-19,10.0,NaN,NaN,NaN,NaN
4,4,46272,309,2016-12-19,15.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,NaN,0.0,1000.00,1000.0,1.0
35340,35538,40372,1380,2019-12-30,NaN,0.0,2000.00,2000.0,1.0
35341,35539,40373,1380,2019-12-30,NaN,0.0,3000.00,3000.0,1.0
35342,35540,46272,1380,2019-12-30,NaN,1.0,284.29,199.0,1.0


In [7]:
# investigate data holiscitly
train.describe()

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
count,35344.000000,35344.000000,35344.000000,35344,34144.000000,35159.000000,34217.000000,34212.000000,35159.000000
mean,17766.554012,49253.732232,844.240154,2018-07-24 16:17:19.746491904,12.245636,0.206434,1167.679357,1155.778351,0.907677
min,0.000000,40369.000000,309.000000,2016-12-19 00:00:00,0.000000,0.000000,49.000000,8.647059,0.000000
25%,8881.750000,40370.000000,535.000000,2017-11-06 00:00:00,2.000000,0.000000,284.290000,199.000000,1.000000
50%,17770.500000,40372.000000,862.000000,2018-07-30 00:00:00,6.000000,0.000000,1000.000000,1000.000000,1.000000
75%,26647.250000,46272.000000,1173.000000,2019-04-22 00:00:00,12.000000,0.000000,2000.000000,2000.000000,1.000000
max,35541.000000,96212.000000,1380.000000,2019-12-30 00:00:00,1160.000000,2.000000,3000.000000,3000.000000,1.000000
std,10258.040738,19145.064867,333.229160,NaN,32.604642,0.433393,1046.828551,1057.912830,0.289486


In [8]:
# draw some time series
def plot_some_ts(ts_df, groupby_columns, time_column, target_column, ts_num = 10, aggregation_method = 'sum'):
  '''pivot original ts by group_columns
     ts_df - original dataframe with ts data,
     group_columns - tuple with names of columns used to split data to time series, use None if splitting is not needed
     time_column - name of column with date, datetime64
     target_column - column with ts data (data should be in numerical format), string
     ts_num - number of ts to be drawn, int
     aggregation_method - aggregation method of data in target column, string
  '''

  if groupby_columns is None:
    ts_df = ts_df.copy()
    ts_df['_temp_const'] = 1
    groupby_columns = ['_temp_const']

  pivot_ts = ts_df.groupby(groupby_columns + [time_column]).agg(aggregation_method)

  # concat multiple index to single column
  index_column_name = ', '.join([groupby_columns[i]+'={0['+str(i)+']}' for i in range(len(groupby_columns))])
  pivot_ts.index = [pivot_ts.index.map(index_column_name.format) , pivot_ts.index.get_level_values(len(groupby_columns))]

  # unstack by-column (column that contains ts name)
  pivot_ts = pivot_ts.unstack([0])[target_column]

  # plot first ts_num ts
  fig = pivot_ts[pivot_ts.columns[:ts_num]].plot().update_layout(height=350, width=1300,
                                                  title="first {0} ts for {1} variable".format(ts_num, target_column ),
    xaxis_title=time_column,
    yaxis_title=target_column+ ' value',
    legend_title='ts id columns: '+', '.join(groupby_columns)).show()
  return fig

# data for 3 products (all stores-product level)
plot_some_ts(train, ['product_rk'], 'period_start_dt', 'demand', ts_num = 6)


In [9]:
# all stores - all products level
# pay attention to forecasting periods: since 2Dec2019
plot_some_ts(train, None, 'period_start_dt', 'demand', ts_num = 1)

# what data dependencies can you observe?

Here

# data preprocessing
 - do somethin with missing values
 - add some demand drivers to dataset

## fill missing values based on expert insights

In [10]:
# fill na in PROMO1_FLAG with mode-value (based on common-sense)
train['PROMO1_FLAG'] = train['PROMO1_FLAG'].fillna(train['PROMO1_FLAG'].mode()[0]) # most frequent value
train

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
0,0,40369,309,2016-12-19,29.0,0.0,NaN,NaN,NaN
1,1,40370,309,2016-12-19,64.0,0.0,NaN,NaN,NaN
2,2,40372,309,2016-12-19,32.0,0.0,NaN,NaN,NaN
3,3,40373,309,2016-12-19,10.0,0.0,NaN,NaN,NaN
4,4,46272,309,2016-12-19,15.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,NaN,0.0,1000.00,1000.0,1.0
35340,35538,40372,1380,2019-12-30,NaN,0.0,2000.00,2000.0,1.0
35341,35539,40373,1380,2019-12-30,NaN,0.0,3000.00,3000.0,1.0
35342,35540,46272,1380,2019-12-30,NaN,1.0,284.29,199.0,1.0


In [11]:
train['PROMO1_FLAG'].unique()

array([0., 1., 2.])

In [12]:
# If 2.0 exists and should be treated as 1.0 (promo active), fix it:
train.loc[train['PROMO1_FLAG'] == 2.0, 'PROMO1_FLAG'] = 1.0

## fill missing values with prev/back info

In [13]:
# AUTORIZATION_FLAG - is product available at store at the moment
plot_some_ts(train, ['product_rk', 'store_location_rk'], 'period_start_dt', 'AUTORIZATION_FLAG', ts_num = 3)

In [14]:
# filling withh prev then next value in pandas
train.set_index(['product_rk', 'store_location_rk', 'period_start_dt'])\
  .unstack([0,1])\
   ['PRICE_REGULAR'].\
   ffill().bfill().\
   stack([1,0], future_stack=True)\
   .rename('REGULAR_PRICE_FIXED')

period_start_dt  store_location_rk  product_rk
2016-12-19       309                40369             NaN
                                    40370             NaN
                                    40372             NaN
                                    40373             NaN
                                    46272             NaN
                                                   ...   
2019-12-30       1380               40370         1000.00
                                    40372         2000.00
                                    40373         3000.00
                                    96212          141.43
                                    46272          284.29
Name: REGULAR_PRICE_FIXED, Length: 38955, dtype: float64

In [15]:
# fill na with prev (and if no prev then next) values
train = train.set_index(['product_rk', 'store_location_rk', 'period_start_dt']).\
  merge( train.set_index(['product_rk', 'store_location_rk', 'period_start_dt'])\
            .unstack([0,1])\
            ['PRICE_REGULAR'].\
              ffill().bfill().\
            stack([1,0],  future_stack=True).\
            rename('PRICE_REGULAR_FIXED'),
         how = 'left', right_index = True, left_index = True)\
  .reset_index()

# dell original column and replace with new one
del train['PRICE_REGULAR']
train.rename(columns = {'PRICE_REGULAR_FIXED':'PRICE_REGULAR'}, inplace=True)
train.head()

,product_rk,store_location_rk,period_start_dt,id,demand,PROMO1_FLAG,PRICE_AFTER_DISC,AUTORIZATION_FLAG,PRICE_REGULAR
0,40369,309,2016-12-19,0,29.0,0.0,NaN,NaN,NaN
1,40370,309,2016-12-19,1,64.0,0.0,NaN,NaN,NaN
2,40372,309,2016-12-19,2,32.0,0.0,NaN,NaN,NaN
3,40373,309,2016-12-19,3,10.0,0.0,NaN,NaN,NaN
4,46272,309,2016-12-19,4,15.0,0.0,NaN,NaN,NaN


In [16]:
def ts_fillna_ffill_bfill(ts_df,column_name, ts_id):
#  all_data.set_index(['product_rk', 'store_location_rk', 'period_start_dt'])\ # define id columns
#   .unstack([0,1])\                          # df -> pivot transformation: date column - is row-index, product x store - is column index
#   ['PRICE_REGULAR'].\                      # define column with data to be fixed
#   ffill().bfill().\                        # fill missing value: apply forwand then back filling method consequently
#   stack([1,0]).\                           # pivot -> ts transformation
#   rename('REGULAR_PRICE_FIXED')            # rename column


  # fill na with prev (and if no prev then next) values
  new_ts_df = ts_df.set_index(ts_id).\
    merge(ts_df.set_index(ts_id)\
              .unstack([0,1])\
              [column_name].\
              ffill().bfill().\
              stack([1,0],  future_stack=True).\
              rename(column_name),
          how = 'left', right_index = True, left_index = True)\
    .reset_index()

  # dell original column and replace with new one
  del new_ts_df[column_name+'_x']
  return new_ts_df.rename(columns = {column_name+'_y':column_name})

In [17]:
train.head()

,product_rk,store_location_rk,period_start_dt,id,demand,PROMO1_FLAG,PRICE_AFTER_DISC,AUTORIZATION_FLAG,PRICE_REGULAR
0,40369,309,2016-12-19,0,29.0,0.0,NaN,NaN,NaN
1,40370,309,2016-12-19,1,64.0,0.0,NaN,NaN,NaN
2,40372,309,2016-12-19,2,32.0,0.0,NaN,NaN,NaN
3,40373,309,2016-12-19,3,10.0,0.0,NaN,NaN,NaN
4,46272,309,2016-12-19,4,15.0,0.0,NaN,NaN,NaN


In [18]:
# the same fort PRICE_AFTER_DISCOUNT
train = ts_fillna_ffill_bfill(ts_df = train,column_name = 'PRICE_AFTER_DISC' , ts_id= ['product_rk', 'store_location_rk', 'period_start_dt'])


# the same fort AUTORIZATION_FLAG
train = ts_fillna_ffill_bfill(ts_df = train,column_name = 'AUTORIZATION_FLAG' , ts_id= ['product_rk', 'store_location_rk', 'period_start_dt'])

In [19]:
# look at data again
train.head()

,product_rk,store_location_rk,period_start_dt,id,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
0,40369,309,2016-12-19,0,29.0,0.0,NaN,NaN,NaN
1,40370,309,2016-12-19,1,64.0,0.0,NaN,NaN,NaN
2,40372,309,2016-12-19,2,32.0,0.0,NaN,NaN,NaN
3,40373,309,2016-12-19,3,10.0,0.0,NaN,NaN,NaN
4,46272,309,2016-12-19,4,15.0,0.0,NaN,NaN,NaN


## fill missing values as average/mode/median from other stores

In [20]:
# find mean values for each pair product x date
values = train.set_index(['product_rk', 'period_start_dt', 'store_location_rk'])\
            .unstack([0,1])\
            ['PRICE_REGULAR'].\
              mean()

# replace missing values with mean in all stores
train.set_index(['product_rk', 'store_location_rk', 'period_start_dt'])\
            .unstack([0,2])\
            ['PRICE_REGULAR'].\
            fillna(value = values).\
            stack([1,0], future_stack=True).\
            rename('REGULAR_PRICE_FIXED')

store_location_rk  period_start_dt  product_rk
309                2016-12-19       40369          500.00
                                    40370         1000.00
                                    40372         2000.00
                                    40373         3000.00
                                    46272          157.00
                                                   ...   
1380               2019-12-30       40372         2000.00
                                    40373         3000.00
                                    46272          284.29
                                    96212          141.43
                   2017-07-17       96212           79.00
Name: REGULAR_PRICE_FIXED, Length: 37884, dtype: float64

In [21]:
def ts_fillna_aggmethod(ts_df,column_name, ts_id):
#  all_data.set_index(['product_rk', 'store_location_rk', 'period_start_dt'])\ # define id columns
#   .unstack([0,1])\                          # df -> pivot transformation: date column - is row-index, product x store - is column index
#   ['PRICE_REGULAR'].\                      # define column with data to be fixed
#   ffill().bfill().\                        # fill missing value: apply forwand then back filling method consequently
#   stack([1,0]).\                           # pivot -> ts transformation
#   rename('REGULAR_PRICE_FIXED')            # rename column

  values = train.set_index(ts_id)\
            .unstack([0,1])\
            [column_name].\
              mean()

  # fill na with prev (and if no prev then next) values
  new_ts_df = ts_df.set_index(ts_id).\
    merge(ts_df.set_index(ts_id)\
              .unstack([0,1])\
              [column_name].\
              fillna(value = values).\
              stack([1,0], future_stack=True).\
              rename(column_name),
          how = 'left', right_index = True, left_index = True)\
    .reset_index()

  # dell original column and replace with new one
  del new_ts_df[column_name+'_x']
  return new_ts_df.rename(columns = {column_name+'_y':column_name})


# for PRICE_REGULAR
train = ts_fillna_aggmethod(ts_df = train,column_name = 'PRICE_REGULAR' , ts_id= ['product_rk', 'period_start_dt', 'store_location_rk'])


In [22]:
# the same fort PRICE_AFTER_DISCOUNT
train = ts_fillna_aggmethod(ts_df = train,column_name = 'PRICE_AFTER_DISC' , ts_id= ['product_rk', 'period_start_dt', 'store_location_rk'])


# the same fort AUTORIZATION_FLAG
train = ts_fillna_aggmethod(ts_df = train,column_name = 'AUTORIZATION_FLAG' , ts_id= ['product_rk', 'period_start_dt', 'store_location_rk'])

In [23]:
# check data again
train.isna().sum()

,0
product_rk,0
period_start_dt,0
store_location_rk,0
id,0
demand,1200
PROMO1_FLAG,0
PRICE_REGULAR,0
PRICE_AFTER_DISC,0
AUTORIZATION_FLAG,0


In [24]:
# let's delete store 309 related data
train = train[train['store_location_rk'] != 309]
train

,product_rk,period_start_dt,store_location_rk,id,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
15,40369,2016-12-19,317,15,50.0,0.0,500.00,500.0,1.0
16,40370,2016-12-19,317,16,44.0,0.0,1000.00,1000.0,1.0
17,40372,2016-12-19,317,17,13.0,0.0,2000.00,2000.0,1.0
18,40373,2016-12-19,317,18,6.0,0.0,3000.00,3000.0,1.0
19,46272,2016-12-19,317,19,34.0,0.0,157.00,157.0,1.0
...,...,...,...,...,...,...,...,...,...
35339,40370,2019-12-30,1380,35537,NaN,0.0,1000.00,1000.0,1.0
35340,40372,2019-12-30,1380,35538,NaN,0.0,2000.00,2000.0,1.0
35341,40373,2019-12-30,1380,35539,NaN,0.0,3000.00,3000.0,1.0
35342,46272,2019-12-30,1380,35540,NaN,1.0,284.29,199.0,1.0


In [25]:
# check data again
# that's it
train.isna().sum()

,0
product_rk,0
period_start_dt,0
store_location_rk,0
id,0
demand,1200
PROMO1_FLAG,0
PRICE_REGULAR,0
PRICE_AFTER_DISC,0
AUTORIZATION_FLAG,0


##Create discount percentage feature

In [26]:
train['DISCOUNT_PCT'] = ((train['PRICE_REGULAR'] - train['PRICE_AFTER_DISC']) /
                         train['PRICE_REGULAR'] * 100)
train['DISCOUNT_PCT'] = train['DISCOUNT_PCT'].fillna(0)

## add calendar-feature

In [27]:
# all_data['period_start_dt'] = pd.to_datetime(all_data['period_start_dt'], dayfirst =False)
train["ind_of_year"] = [dt.year for dt in train.period_start_dt]
train["ind_of_month"] = [dt.month for dt in train.period_start_dt]
train["ind_of_day"] = [dt.day for dt in train.period_start_dt]

# Add more useful calendar features
train['ind_of_week'] = train['period_start_dt'].dt.isocalendar().week
train['ind_of_quarter'] = train['period_start_dt'].dt.quarter
train['is_month_start'] = train['period_start_dt'].dt.is_month_start.astype(int)
train['is_month_end'] = train['period_start_dt'].dt.is_month_end.astype(int)

train

,product_rk,period_start_dt,store_location_rk,id,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,DISCOUNT_PCT,ind_of_year,ind_of_month,ind_of_day,ind_of_week,ind_of_quarter,is_month_start,is_month_end
15,40369,2016-12-19,317,15,50.0,0.0,500.00,500.0,1.0,0.000000,2016,12,19,51,4,0,0
16,40370,2016-12-19,317,16,44.0,0.0,1000.00,1000.0,1.0,0.000000,2016,12,19,51,4,0,0
17,40372,2016-12-19,317,17,13.0,0.0,2000.00,2000.0,1.0,0.000000,2016,12,19,51,4,0,0
18,40373,2016-12-19,317,18,6.0,0.0,3000.00,3000.0,1.0,0.000000,2016,12,19,51,4,0,0
19,46272,2016-12-19,317,19,34.0,0.0,157.00,157.0,1.0,0.000000,2016,12,19,51,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35339,40370,2019-12-30,1380,35537,NaN,0.0,1000.00,1000.0,1.0,0.000000,2019,12,30,1,4,0,0
35340,40372,2019-12-30,1380,35538,NaN,0.0,2000.00,2000.0,1.0,0.000000,2019,12,30,1,4,0,0
35341,40373,2019-12-30,1380,35539,NaN,0.0,3000.00,3000.0,1.0,0.000000,2019,12,30,1,4,0,0
35342,46272,2019-12-30,1380,35540,NaN,1.0,284.29,199.0,1.0,30.001055,2019,12,30,1,4,0,0


In [28]:
train.describe()

,product_rk,period_start_dt,store_location_rk,id,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,DISCOUNT_PCT,ind_of_year,ind_of_month,ind_of_day,ind_of_week,ind_of_quarter,is_month_start,is_month_end
count,35329.000000,35329,35329.000000,35329.00000,34129.000000,35329.000000,35329.000000,35329.000000,35329.000000,35329.000000,35329.000000,35329.000000,35329.000000,35329.0,35329.000000,35329.000000,35329.000000
mean,49257.002576,2018-07-24 22:09:17.796711168,844.467406,17774.09437,12.226552,0.193495,1167.325471,1155.643781,0.908064,5.828839,2018.048232,6.700359,15.725664,27.061621,2.573891,0.031787,0.026041
min,40369.000000,2016-12-19 00:00:00,317.000000,15.00000,0.000000,0.000000,49.000000,8.647059,0.000000,0.000000,2016.000000,1.000000,1.000000,1.0,1.000000,0.000000,0.000000
25%,40370.000000,2017-11-13 00:00:00,535.000000,8893.00000,2.000000,0.000000,284.290000,199.000000,1.000000,0.000000,2017.000000,4.000000,8.000000,14.0,2.000000,0.000000,0.000000
50%,40372.000000,2018-07-30 00:00:00,862.000000,17778.00000,6.000000,0.000000,1000.000000,1000.000000,1.000000,0.000000,2018.000000,7.000000,16.000000,27.0,3.000000,0.000000,0.000000
75%,46272.000000,2019-04-22 00:00:00,1173.000000,26651.00000,12.000000,0.000000,2000.000000,2000.000000,1.000000,0.000000,2019.000000,10.000000,23.000000,40.0,4.000000,0.000000,0.000000
max,96212.000000,2019-12-30 00:00:00,1380.000000,35541.00000,1160.000000,1.000000,3000.000000,3000.000000,1.000000,87.647059,2019.000000,12.000000,31.000000,52.0,4.000000,1.000000,1.000000
std,19148.409051,NaN,333.117293,10253.68736,32.585194,0.395043,1047.244561,1057.981350,0.288939,14.490145,0.821643,3.494496,8.803124,15.27552,1.120176,0.175435,0.159259


# train ML model

In [29]:
from ipywidgets import IntProgress

from itertools import product
def percentile(n):
    '''Calculate n - percentile of data'''
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'pctl%s' % n
    return percentile_

# add missing dates to GroupBy.Core object
def fill_missing_dates(x, date_col, freq = None, default_value = np.nan):

    if freq is None:
        try:
           freq = pd.infer_freq(x.set_index(date_col).index[:min(100, x.shape[0])])
        except:
           freq = 'D'

        if freq is None:
          freq = 'D'
          Warning('TS freq is not defined! Daily granularity is provided!')
    # print(freq)
    # new indexes without time breaks
    idx = pd.date_range(x[date_col].min(), x[date_col].max(), freq=freq)
    # print(idx)
    results = x.set_index(date_col).reindex(idx,fill_value = default_value)
    results.index.rename(date_col, inplace = True)

    # groupby_day = x.groupby(pd.PeriodIndex(x[date_col], freq='D'))
    # results = groupby_day.sum(min_count=1)

    # idx = pd.period_range(min_date, max_date)
    # results = results.reindex(idx, fill_value=default_value)

    # results.index.rename(date_col, inplace=True)

    return results.reset_index()


def calc_preag_fill(data, group_col, date_col, target_cols, preagg_method):
    ## calc preaggregation
    data_preag = data.groupby(group_col).agg(
        preagg_method)[target_cols].reset_index()

    ## fill missing dates
    data_preag_filled = data_preag.groupby(group_col[:-1]).apply(
         fill_missing_dates, date_col=date_col).drop(group_col[:-1],
                                                     axis=1).reset_index()

    ## return DataFrame with calculated preaggregation and filled missing dates
    return data_preag # ,  data_preag_filled


def calc_rolling(data_preag_filled, group_col, date_col, method, w):

    ## calc rolling stats
    lf_df_filled = data_preag_filled.groupby(group_col[:-1]).\
        apply(lambda x: x.set_index(date_col).rolling(window=w, min_periods=1).agg(method)).drop(group_col[:-1], axis=1).reset_index(group_col)

    ## return DataFrame with rolled columns from target_vars
    return lf_df_filled

# ewma calculation method
def calc_ewm(data_preag_filled, group_col, date_col, span):
    ## calc ewm stats
    lf_df_filled = data_preag_filled.groupby(group_col[:-1]).\
        apply(lambda x: x.set_index(date_col).ewm(span=span).mean()).drop(group_col[:-1], axis=1).reset_index(group_col)

    ## return DataFrame with rolled columns from target_vars
    return lf_df_filled

# shift ts data
def shift(lf_df_filled, group_col, date_col, lag, kwargs = None):

    lf_df = (lf_df_filled.     # prepare calculations
        set_index(date_col).    # date column as time-index
        groupby(group_col[:-1]).       # ids_col to separate different time series
        apply(lambda x: x.shift(lag, kwargs)).    # aplly shift at lag steps
        drop(group_col[:-1], axis=1).                   # drop ids_col from time df to reset index
        reset_index()              # reset index to return df to no-index state
    )
    # lf_df[date_col] = pd.to_datetime(lf_df[date_col].astype(str))
    # print(lf_df.index)

    ## return DataFrame with following columns: filter_col, id_cols, date_col and shifted stats
    return lf_df


def generate_lagged_features(
        data: pd.DataFrame,
        target_cols: list = ['Demand'],
        id_cols: list = ['SKU_id', 'Store_id'],
        date_col: str = 'Date',
        lags: list = [7, 14, 21, 28],
        windows: list = ['7D', '14D', '28D', '56D'],
        preagg_methods: list = ['mean'],
        agg_methods: list = ['mean', 'median', percentile(10), pd.Series.skew],
        dynamic_filters: list = ['weekday', 'Promo'],
        ewm_params: dict = {'weekday': [14, 28], 'Promo': [14, 42]}) -> pd.DataFrame:

    '''
    data - dataframe with default index
    target_cols - column names for lags calculation
    id_cols - key columns to identify unique values
    date_col - column with datetime format values
    lags - lag values(days)
    windows - windows(days/weeks/months/etc.),
        calculation is performed within time range length of window
    preagg_methods - applied methods before rolling to make
        every value unique for given id_cols
    agg_methods - method of aggregation('mean', 'median', percentile, etc.)
    dynamic_filters - column names to use as filter
    ewm_params - span values(days) for each dynamic_filter
    '''

    data = data.sort_values(date_col)
    out_df = deepcopy(data)
    dates = [min(data[date_col]), max(data[date_col])]

    total = len(target_cols) * len(lags) * len(windows) * len(preagg_methods) * len(agg_methods) * len(dynamic_filters)
    progress = IntProgress(min=0, max=total)
    display(progress)

    for filter_col in dynamic_filters:
        group_col = [filter_col] + id_cols + [date_col]
        for preagg in preagg_methods:
          data_preag_filled = calc_preag_fill(data, group_col, date_col,
                                                  target_cols, preagg)

          ## add ewm features
          for alpha in ewm_params.get(filter_col, []):
              ewm_filled = calc_ewm(data_preag_filled, group_col,
                                    date_col, alpha)
              for lag in lags:
                ewm = shift(ewm_filled, group_col, date_col, lag)

                new_names = {x: "{0}_lag{1}d_alpha{2}_key{3}_preag{4}_{5}_dynamic_ewm".\
                    format(x, lag, alpha, '&'.join(id_cols), preagg, filter_col) for x in target_cols}

                out_df = pd.merge(out_df,
                                  ewm.rename(columns=new_names),
                                  how='left',
                                  on=group_col)

          ## add rolling features
          for w in windows:
              for method in agg_methods:
                  rolling_filled = calc_rolling(data_preag_filled,
                                                group_col, date_col,
                                                method, w)
                  for lag in lags:
                    ## lf_df - DataFrame with following columns: filter_col, id_cols, date_col, shifted rolling stats
                    rolling = shift(rolling_filled, group_col, date_col, lag)

                    method_name = method.__name__ if type(
                        method) != str else method

                    new_names = {x: "{0}_lag{1}d_w{2}_key{3}_preag{4}_ag{5}_{6}_dynamic_rolling".\
                                  format(x, lag, w, '&'.join(id_cols), preagg, method_name, filter_col) for x in target_cols}

                    out_df = pd.merge(out_df,
                                      rolling.rename(columns=new_names),
                                      how='left',
                                      on=group_col)
                    progress.value += 1

    return out_df

In [30]:
target_cols = ['demand']
id_cols = ['product_rk', 'store_location_rk']
date_col = 'period_start_dt'

train['NoFilter'] = 1

train = generate_lagged_features(
    train,
    target_cols=['demand'],
    id_cols=['product_rk', 'store_location_rk'],
    date_col='period_start_dt',
    lags=[7, 14, 21, 28],  # Min lag >= forecast horizon (7 days)
    windows=['7D', '14D', '28D'],
    preagg_methods=['mean', 'sum'],
    agg_methods=['mean', 'median', percentile(10), percentile(90)],
    dynamic_filters=['NoFilter', 'PROMO1_FLAG'],
    ewm_params={'NoFilter': [14, 28], 'PROMO1_FLAG': [14, 28]}
)

IntProgress(value=0, max=192)

/tmp/ipython-input-2762051727.py:47: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

/tmp/ipython-input-2762051727.py:68: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

/tmp/ipython-input-2762051727.py:79: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either p

In [31]:
test.describe()

,id,product_rk,store_location_rk,demand
count,1404.000000,1404.000000,1404.000000,0.0
mean,18922.480769,50466.672365,866.433048,NaN
std,10597.300379,20317.430200,343.378421,NaN
min,908.000000,40369.000000,317.000000,NaN
25%,9293.750000,40370.000000,535.000000,NaN
50%,18576.500000,40372.000000,862.000000,NaN
75%,28750.250000,46272.000000,1191.000000,NaN
max,35547.000000,96212.000000,1380.000000,NaN


In [35]:
#split train and test data
data_train = train[train['demand'].isnull() == False]
data_train = data_train.fillna(0)
data_train

,product_rk,period_start_dt,store_location_rk,id,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,DISCOUNT_PCT,...,demand_lag21d_w28D_keyproduct_rk&store_location_rk_preagsum_agmedian_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_rk&store_location_rk_preagsum_agmedian_PROMO1_FLAG_dynamic_rolling,demand_lag7d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag14d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag21d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag7d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag14d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag21d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling
0,40369,2016-12-19,317,15,50.0,0.0,500.00,500.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,40369,2016-12-19,1034,21361,20.0,0.0,500.00,500.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,40370,2016-12-19,1034,21362,31.0,0.0,1000.00,1000.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,40372,2016-12-19,1034,21363,11.0,0.0,2000.00,2000.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,40373,2016-12-19,1034,21364,14.0,0.0,3000.00,3000.0,1.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34124,46272,2019-11-25,1034,22253,3.0,1.0,284.29,199.0,1.0,30.001055,...,7.0,5.0,0.3,6.3,2.8,3.3,2.7,9.8,8.4,13.0
34125,40373,2019-11-25,1034,22252,6.0,0.0,3000.00,3000.0,1.0,0.000000,...,2.0,3.5,0.6,4.3,0.6,0.9,4.4,7.0,6.9,4.7
34126,96212,2019-11-25,1202,29678,0.0,0.0,49.00,49.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34127,40369,2019-11-25,1034,22249,12.0,0.0,500.00,500.0,1.0,0.000000,...,3.0,1.5,0.3,0.0,0.6,0.3,4.0,7.9,4.0,2.7


In [36]:
data_test = train[train['demand'].isnull()]
# rename "demand" to "predicted"
data_test.rename(columns={'demand': 'predicted'}, inplace=True)
data_test # 1200 samples

/tmp/ipython-input-4079278133.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,product_rk,period_start_dt,store_location_rk,id,predicted,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,DISCOUNT_PCT,...,demand_lag21d_w28D_keyproduct_rk&store_location_rk_preagsum_agmedian_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_rk&store_location_rk_preagsum_agmedian_PROMO1_FLAG_dynamic_rolling,demand_lag7d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag14d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag21d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag7d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag14d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag21d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling
34129,40369,2019-12-02,1143,25034,NaN,0.0,500.00,500.0,1.0,0.000000,...,0.000000,2.500000,0.0,0.000000,0.000000,0.600000,2.1,3.100000,2.100000,4.400000
34130,40370,2019-12-02,1143,25035,NaN,0.0,1000.00,1000.0,1.0,0.000000,...,0.000000,3.500000,0.0,0.300000,0.000000,0.600000,5.1,2.000000,3.500000,5.000000
34131,40370,2019-12-02,1079,23184,NaN,0.0,1000.00,1000.0,1.0,0.000000,...,8.000000,6.000000,0.0,10.000000,2.100000,1.800000,10.8,16.000000,12.500000,8.800000
34132,96212,2019-12-02,1005,21336,NaN,0.0,99.00,69.3,0.0,30.000000,...,0.000000,46.000000,0.0,0.000000,0.000000,42.000000,0.0,0.000000,0.000000,50.000000
34133,40372,2019-12-02,525,7409,NaN,0.0,2000.00,2000.0,1.0,0.000000,...,10.404372,10.404372,0.0,10.404372,3.121312,3.121312,17.6,17.083607,10.404372,10.404372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35324,40369,2019-12-30,637,13921,NaN,0.0,500.00,500.0,1.0,0.000000,...,2.500000,5.000000,0.9,1.600000,0.600000,0.600000,8.1,9.800000,7.200000,8.000000
35325,40370,2019-12-30,637,13922,NaN,0.0,1000.00,1000.0,1.0,0.000000,...,14.000000,15.500000,5.6,10.000000,8.500000,4.500000,14.6,19.700000,16.700000,18.800000
35326,40372,2019-12-30,637,13923,NaN,0.0,2000.00,2000.0,1.0,0.000000,...,10.000000,10.000000,6.9,5.300000,8.600000,1.500000,10.7,8.100000,11.400000,15.700000
35327,40372,2019-12-30,557,12065,NaN,0.0,2000.00,2000.0,1.0,0.000000,...,7.500000,10.500000,4.6,2.900000,6.300000,2.100000,10.1,9.500000,12.200000,16.100000


In [37]:
X = data_train.drop(['id', 'demand', 'period_start_dt'], axis=1)
y = data_train['demand']

In [38]:
X

,product_rk,store_location_rk,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,DISCOUNT_PCT,ind_of_year,ind_of_month,ind_of_day,...,demand_lag21d_w28D_keyproduct_rk&store_location_rk_preagsum_agmedian_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_rk&store_location_rk_preagsum_agmedian_PROMO1_FLAG_dynamic_rolling,demand_lag7d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag14d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag21d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag7d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag14d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag21d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling
0,40369,317,0.0,500.00,500.0,1.0,0.000000,2016,12,19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,40369,1034,0.0,500.00,500.0,1.0,0.000000,2016,12,19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,40370,1034,0.0,1000.00,1000.0,1.0,0.000000,2016,12,19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,40372,1034,0.0,2000.00,2000.0,1.0,0.000000,2016,12,19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,40373,1034,0.0,3000.00,3000.0,1.0,0.000000,2016,12,19,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34124,46272,1034,1.0,284.29,199.0,1.0,30.001055,2019,11,25,...,7.0,5.0,0.3,6.3,2.8,3.3,2.7,9.8,8.4,13.0
34125,40373,1034,0.0,3000.00,3000.0,1.0,0.000000,2019,11,25,...,2.0,3.5,0.6,4.3,0.6,0.9,4.4,7.0,6.9,4.7
34126,96212,1202,0.0,49.00,49.0,0.0,0.000000,2019,11,25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34127,40369,1034,0.0,500.00,500.0,1.0,0.000000,2019,11,25,...,3.0,1.5,0.3,0.0,0.6,0.3,4.0,7.9,4.0,2.7


In [39]:
# answers in train period
y

,demand
0,50.0
1,20.0
2,31.0
3,11.0
4,14.0
...,...
34124,3.0
34125,6.0
34126,0.0
34127,12.0


In [40]:
# for training
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [41]:
# train model
regressor = GradientBoostingRegressor(
    max_depth=11,
    n_estimators=420,
    learning_rate=0.2,
    random_state=1,
    min_samples_leaf=11,
    min_samples_split=2,
    loss='absolute_error'
)
regressor.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.2, loss='absolute_error',
                          max_depth=11, min_samples_leaf=11, n_estimators=420,
                          random_state=1)

In [42]:
y_pred = regressor.predict(X_test)
mean_absolute_error(y_test, y_pred)

5.5051260085391425

In [43]:
# score sample from forecasting period
X_test = data_test.drop(['id', 'predicted', 'period_start_dt'], axis=1)
X_test

,product_rk,store_location_rk,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,DISCOUNT_PCT,ind_of_year,ind_of_month,ind_of_day,...,demand_lag21d_w28D_keyproduct_rk&store_location_rk_preagsum_agmedian_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_rk&store_location_rk_preagsum_agmedian_PROMO1_FLAG_dynamic_rolling,demand_lag7d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag14d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag21d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag7d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag14d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag21d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling
34129,40369,1143,0.0,500.00,500.0,1.0,0.000000,2019,12,2,...,0.000000,2.500000,0.0,0.000000,0.000000,0.600000,2.1,3.100000,2.100000,4.400000
34130,40370,1143,0.0,1000.00,1000.0,1.0,0.000000,2019,12,2,...,0.000000,3.500000,0.0,0.300000,0.000000,0.600000,5.1,2.000000,3.500000,5.000000
34131,40370,1079,0.0,1000.00,1000.0,1.0,0.000000,2019,12,2,...,8.000000,6.000000,0.0,10.000000,2.100000,1.800000,10.8,16.000000,12.500000,8.800000
34132,96212,1005,0.0,99.00,69.3,0.0,30.000000,2019,12,2,...,0.000000,46.000000,0.0,0.000000,0.000000,42.000000,0.0,0.000000,0.000000,50.000000
34133,40372,525,0.0,2000.00,2000.0,1.0,0.000000,2019,12,2,...,10.404372,10.404372,0.0,10.404372,3.121312,3.121312,17.6,17.083607,10.404372,10.404372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35324,40369,637,0.0,500.00,500.0,1.0,0.000000,2019,12,30,...,2.500000,5.000000,0.9,1.600000,0.600000,0.600000,8.1,9.800000,7.200000,8.000000
35325,40370,637,0.0,1000.00,1000.0,1.0,0.000000,2019,12,30,...,14.000000,15.500000,5.6,10.000000,8.500000,4.500000,14.6,19.700000,16.700000,18.800000
35326,40372,637,0.0,2000.00,2000.0,1.0,0.000000,2019,12,30,...,10.000000,10.000000,6.9,5.300000,8.600000,1.500000,10.7,8.100000,11.400000,15.700000
35327,40372,557,0.0,2000.00,2000.0,1.0,0.000000,2019,12,30,...,7.500000,10.500000,4.6,2.900000,6.300000,2.100000,10.1,9.500000,12.200000,16.100000


In [44]:
# look at forecsting values
y_pred_res = regressor.predict(X_test)
y_pred_res

array([ 1.64392884,  3.03568706, 10.76827101, ..., 28.93516595,
       28.71374726, 16.44287835])

# forecast postprocessing

In [52]:
data_test

,product_rk,period_start_dt,store_location_rk,id,predicted,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,DISCOUNT_PCT,...,demand_lag21d_w28D_keyproduct_rk&store_location_rk_preagsum_agmedian_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_rk&store_location_rk_preagsum_agmedian_PROMO1_FLAG_dynamic_rolling,demand_lag7d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag14d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag21d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl10_PROMO1_FLAG_dynamic_rolling,demand_lag7d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag14d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag21d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling,demand_lag28d_w28D_keyproduct_rk&store_location_rk_preagsum_agpctl90_PROMO1_FLAG_dynamic_rolling
34129,40369,2019-12-02,1143,25034,NaN,0.0,500.00,500.0,1.0,0.000000,...,0.000000,2.500000,0.0,0.000000,0.000000,0.600000,2.1,3.100000,2.100000,4.400000
34130,40370,2019-12-02,1143,25035,NaN,0.0,1000.00,1000.0,1.0,0.000000,...,0.000000,3.500000,0.0,0.300000,0.000000,0.600000,5.1,2.000000,3.500000,5.000000
34131,40370,2019-12-02,1079,23184,NaN,0.0,1000.00,1000.0,1.0,0.000000,...,8.000000,6.000000,0.0,10.000000,2.100000,1.800000,10.8,16.000000,12.500000,8.800000
34132,96212,2019-12-02,1005,21336,NaN,0.0,99.00,69.3,0.0,30.000000,...,0.000000,46.000000,0.0,0.000000,0.000000,42.000000,0.0,0.000000,0.000000,50.000000
34133,40372,2019-12-02,525,7409,NaN,0.0,2000.00,2000.0,1.0,0.000000,...,10.404372,10.404372,0.0,10.404372,3.121312,3.121312,17.6,17.083607,10.404372,10.404372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35324,40369,2019-12-30,637,13921,NaN,0.0,500.00,500.0,1.0,0.000000,...,2.500000,5.000000,0.9,1.600000,0.600000,0.600000,8.1,9.800000,7.200000,8.000000
35325,40370,2019-12-30,637,13922,NaN,0.0,1000.00,1000.0,1.0,0.000000,...,14.000000,15.500000,5.6,10.000000,8.500000,4.500000,14.6,19.700000,16.700000,18.800000
35326,40372,2019-12-30,637,13923,NaN,0.0,2000.00,2000.0,1.0,0.000000,...,10.000000,10.000000,6.9,5.300000,8.600000,1.500000,10.7,8.100000,11.400000,15.700000
35327,40372,2019-12-30,557,12065,NaN,0.0,2000.00,2000.0,1.0,0.000000,...,7.500000,10.500000,4.6,2.900000,6.300000,2.100000,10.1,9.500000,12.200000,16.100000


In [55]:
# convert to df
y_results = data_test[['id', 'predicted']]
y_results['predicted'] = y_pred_res
y_results

/tmp/ipython-input-1605025154.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,predicted
34129,25034,1.643929e+00
34130,25035,3.035687e+00
34131,23184,1.076827e+01
34132,21336,1.191093e-40
34133,7409,1.160261e+01
...,...,...
35324,13921,2.887673e+01
35325,13922,7.867030e+01
35326,13923,2.893517e+01
35327,12065,2.871375e+01


In [46]:
# change negative forecast to 0
y_results.loc[y_results['predicted'] < 0, ['predicted']]

,predicted


In [47]:
y_results.loc[y_results['predicted'] < 0, ['predicted']] = 0
y_results.loc[y_results['predicted'] < 0, ['predicted']]

,predicted


In [48]:
#prepare output csv-file:
y_results.to_csv('./submission_example.csv',sep=',', encoding='utf-8', index=False)

In [56]:
y_results

,id,predicted
34129,25034,1.643929e+00
34130,25035,3.035687e+00
34131,23184,1.076827e+01
34132,21336,1.191093e-40
34133,7409,1.160261e+01
...,...,...
35324,13921,2.887673e+01
35325,13922,7.867030e+01
35326,13923,2.893517e+01
35327,12065,2.871375e+01


In [53]:
! kaggle competitions submit -c dscs-25-hw3 -f submission.csv -m "Message"

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.12/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
